<a href="https://colab.research.google.com/github/tahkootay/ChatGPT_Professional-lesson_17/blob/main/%D0%94%D0%97_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_%E2%84%9617_(%22%D0%94%D0%97_Pro_%7C_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80_%D0%BF%D0%BE_%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE_%7C_ChatGPT_Professional_middle_junior_%7C_%D0%A3%D0%98%D0%98_ipynb%22).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Возьмите видео с ютуб, аудио-файл и текстовый документ по ссылкам, транскрибируйте видео и аудио -дорожку, полученные тексты соедините с текстовым документом, на основе получившегося текста создайте нейро-консультанта для ответов на вопросы по данным материалам, а также краткую методичку.

Видео: https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU Python в вопросах и ответах

Аудио: https://drive.google.com/file/d/1TLsB8EEMqATs18Iwacw5H3nceOFeO7ND/view?usp=sharing ООП в Пайтон

Текст: https://docs.google.com/document/d/1GwrAY4NoqPImiSMola-14ykVB5K-8iDlivWzvuEnMfA/edit?usp=sharing Объяснение алгоритмов сортировки с примерами на Python

P.S. аудио-файлы для обработки whisper стоит сохранить с именем на латинице и без пробелов в названии.

In [ ]:
# указываем ссылку для получения аудио-дорожки из видео
yt_url = 'https://www.youtube.com/watch?v=LRuszWRN4uI&list=PLv_mO3iQ2o2fZllHFsEniuy1D-2IXr8eU'
YouTube_video_title = "Python в вопросах и ответах"

In [ ]:
!pip install git+https://github.com/ytdl-org/youtube-dl.git
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/ytdl-org/youtube-dl.git to /tmp/pip-req-build-2jcfo96p
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-req-build-2jcfo96p
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit e0727e4ab61b6e45f7792546b8b5ff52a0ea22b5
  Preparing metadata (setup.py) ... done
  Created wheel for youtube-dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1955200 sha256=69826b392bff27f01c77de2e24f52edd7cd63a0a9d9f0f24a086517de121cb34
  Stored in directory: /tmp/pip-ephem-wheel-cache-wahgv6f5/wheels/64/8e/40/866e846163e3e4859bbe820ff8847ec97f4320864f1525aa9a
Successfully built youtube-dl
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-l3u2no_m
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-l3u2no_m
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build 

In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 faiss-cpu==1.7.4
!pip install -qq python-docx
!pip install -U pytube
from IPython.display import clear_output
clear_output()

In [ ]:
import whisper
import os
import gdown
import requests
import re
import time
from IPython.display import HTML, clear_output
import subprocess
from pathlib import Path
import json
from pytube import YouTube
import tiktoken
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import ipywidgets as widgets
from IPython.display import display
from tqdm.auto import tqdm
import getpass
import pickle
from urllib.request import urlopen
import openai
import subprocess
import codecs
from langchain.chains import ConversationChain         # Импортируем класс для создания цепочек диалогов
from langchain.chat_models import ChatOpenAI           # Импортируем класс для работы с чатами на базе OpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory  # Импортируем класс для управления памятью диалогов
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import urllib

In [ ]:
# Создает директорию, если она не существует
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Папка для временного хранения скачанных файлов
tmp_folder = '/content/tmp/'
ensure_dir(tmp_folder)

In [ ]:
# yt_url - это переменная, содержащая URL адрес YouTube видео.
# Создаем объект YouTube, который предоставляет интерфейс для взаимодействия с видео по данному URL.
yt = YouTube(yt_url)
# Получаем аудиопоток лучшего качества для данного видео.
# Метод get_audio_only() выбирает из всех доступных потоков тот, который содержит только аудио,
# предпочитая поток с наивысшим качеством.
audio_stream = yt.streams.get_audio_only()
# tmp_folder - переменная, содержащая путь к папке, куда будет сохранен аудиофайл.
# Создаем путь к файлу, используя имя файла по умолчанию, которое предоставляет pytube для скачиваемого аудио.
# При этом заменяем расширение файла с mp4 на m4a, что является более типичным для аудиофайлов,
# хотя оба формата могут использоваться для аудио.
audio_file_path = tmp_folder + audio_stream.default_filename.replace('mp4', 'm4a')
# Скачиваем аудиофайл в указанную папку (tmp_folder) под измененным именем файла (audio_file_path).
# Метод download принимает параметр output_path, указывающий на папку для сохранения,
# и filename, задающий имя файла. Это позволяет сохранить аудио на локальный диск.
audio_stream.download(output_path=tmp_folder, filename=audio_file_path)

'/content/tmp/Изучаем Python  1 глава Python в вопросах и ответах с Алексеем Трояновым.m4a'

In [ ]:
# Используем Whisper для обработки аудиофайла
%%time
!whisper '/content/tmp/Изучаем Python  1 глава Python в вопросах и ответах с Алексеем Трояновым.m4a' --model large --language ru --output_format txt

100%|█████████████████████████████████████| 2.88G/2.88G [01:06<00:00, 46.6MiB/s]
[00:00.440 --> 00:15.080]  Всем привет, добро пожаловать. Этим стулом GAMS начинает новую серию полукресел, а мы этим видео начинаем новую серию видео, она будет называться «Читаем Луца».
[00:15.080 --> 00:41.940]  Когда человек, который начинает учить Python, спрашивает, что же мне почитать, наверное, процентов 90 разработчиков говорят «Читай Луца, там все есть, это такой огромный двухтомник, он жутко нудный, он очень подробный, он достаточно плохо переведен, иногда он очень косноязычен».
[00:41.940 --> 00:44.940]  Но, к сожалению, пожалуй, лучше, чем…
[00:45.080 --> 00:50.140]  Этот учебник просто по базовому Python пока ничего нет.
[00:51.260 --> 01:09.380]  Мы будем читать это дело по главам, на каждую главу у нас будет, скорее всего, отдельный оратор, и он будет давать презентацию, а по ходу вот этой презентации я буду задавать вопросы, каверзные и не очень, ну или подкидывать тему для обсуждения.
[01

In [ ]:
# Задаем путь к файлу
file_path = '/content/tmp/Изучаем Python  1 глава Python в вопросах и ответах с Алексеем Трояновым.txt'

# Открываем файл и читаем его содержимое
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print(f'Файл не найден: {file_path}')